In [48]:
import pandas as pd
df = pd.read_csv('/Users/amroa/Downloads/yelpdata/yelp-csv/yelp_academic_dataset_business.csv')
df.dropna(subset = ['business_id','stars', 'review_count', 'name', 'is_open'], inplace= True)

In [49]:
import numpy as np
df = pd.DataFrame(df[['business_id','attributes']], columns = ['business_id', 'attributes'])
df = df.dropna()
df

,business_id,attributes
0,1SWheh84yJXfytovILXOAQ,{}
1,QXAEGFB4oINsVuTFxEYKFQ,"{'GoodForMeal': ""{'dessert': False, 'latenight..."
2,fcXOEZdXYeZqnQ3lGlOXmg,"{'BusinessParking': ""{'garage': False, 'street..."
3,f2ZWZPENViL92BrFsIgR6w,"{'BusinessParking': ""{'garage': False, 'street..."
4,6KgGE8B1RsR7jc9R5nuH0Q,{}
...,...,...
192601,zsrPJ7WD2OTrH1sbg2S7gg,{}
192603,Ul12enCzNT0G_jj5TCvJ8Q,{}
192604,wWV90q48ugyxyXW6gZav8w,"{'Ambience': ""{'romantic': False, 'intimate': ..."
192605,phSaT3I_y6To1J2OdeFSUQ,{}


In [50]:
filtamb = df['attributes'].str.contains("Music")
df = df[filtamb]
df['attributes']

18        {'NoiseLevel': "'average'", 'Ambience': "{'rom...
56        {'Ambience': "{'romantic': False, 'intimate': ...
90        {'BusinessParking': "{'garage': False, 'street...
93        {'NoiseLevel': "u'average'", 'Ambience': "{'ro...
103       {'Music': "{'dj': False, 'background_music': F...
                                ...                        
192407    {'GoodForMeal': "{'dessert': False, 'latenight...
192445    {'Ambience': "{'touristy': False, 'hipster': F...
192449    {'BusinessParking': "{'garage': False, 'street...
192536    {'Ambience': "{'touristy': False, 'hipster': F...
192594    {'Ambience': "{'touristy': False, 'hipster': F...
Name: attributes, Length: 5249, dtype: object

In [51]:
df.loc[18, 'attributes']

'{\'NoiseLevel\': "\'average\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}"}'

In [52]:
weights = {'dj': 729, 'background_music': 243, 'no_music': 81,
          'jukebox': 27, 'live': 9, 'video': 3, 'karaoke': 1}
def attributeMusicToAid(attributes):
    splitAtAmbience = str(attributes).split("'Music': ")
    beginIndex      = splitAtAmbience[1].find('{')
    endIndex        = splitAtAmbience[1].find('}')
    ambience_fields = (splitAtAmbience[1])[beginIndex:(endIndex+1)]
    splitted_fields = ambience_fields.split(': ')
    
    #trim out the annoying braces
    splitted_fields[0] = (splitted_fields[0])[1:]
    splitted_fields[-1] = (splitted_fields[-1])[0:4]
    if((splitAtAmbience[1]).startswith("'None'")):
        return -1
    
    # get the aid
    aid = 0
    for i in range(0, len(splitted_fields) -1):
        commasplitted1 = splitted_fields[i].split(', ')
        commasplitted2 = splitted_fields[i+1].split(', ')
        name_fields = commasplitted1[-1] #e.g. touristy
        name_value = 2*int(commasplitted2[0] == 'True') + 1*int(commasplitted2[0] == 'False')
        aid = aid + weights[name_fields[1: -1]]*name_value
    return aid

In [53]:
df['attributes'] = df['attributes'].apply(attributeMusicToAid)

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
df

,business_id,attributes
18,8Wm2NMAoukBC2wQR49E4ng,1092
56,uD1dQcPeYIkAPqjzQ79MrQ,1092
90,8jOJmMw6Vfn8ug1nZstH3A,1092
93,LH-_iinkV1XOxSuJMBqhmw,1119
103,rjlAUHT8B6KhKAmkZqS1xw,1092
...,...,...
192407,BF4RaOWyjLDa9OvriKc1sA,1119
192445,h5VZ0MqRt4vEUUG8oH6eEA,1092
192449,6vsvM-v9pNRygE1Z1UMWtg,1011
192536,WoFTWkoOrciT0TEvRJcSTQ,1092


In [55]:
df.replace(-1, np.nan, inplace=True)

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [56]:
df

,business_id,attributes
18,8Wm2NMAoukBC2wQR49E4ng,1092.0
56,uD1dQcPeYIkAPqjzQ79MrQ,1092.0
90,8jOJmMw6Vfn8ug1nZstH3A,1092.0
93,LH-_iinkV1XOxSuJMBqhmw,1119.0
103,rjlAUHT8B6KhKAmkZqS1xw,1092.0
...,...,...
192407,BF4RaOWyjLDa9OvriKc1sA,1119.0
192445,h5VZ0MqRt4vEUUG8oH6eEA,1092.0
192449,6vsvM-v9pNRygE1Z1UMWtg,1011.0
192536,WoFTWkoOrciT0TEvRJcSTQ,1092.0


In [57]:
df = df.dropna()

In [58]:
df.rename(columns = {'attributes': 'MID'}, inplace = True)
df['MID'] = df['MID'].astype('int')
df

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,business_id,MID
18,8Wm2NMAoukBC2wQR49E4ng,1092
56,uD1dQcPeYIkAPqjzQ79MrQ,1092
90,8jOJmMw6Vfn8ug1nZstH3A,1092
93,LH-_iinkV1XOxSuJMBqhmw,1119
103,rjlAUHT8B6KhKAmkZqS1xw,1092
...,...,...
192407,BF4RaOWyjLDa9OvriKc1sA,1119
192445,h5VZ0MqRt4vEUUG8oH6eEA,1092
192449,6vsvM-v9pNRygE1Z1UMWtg,1011
192536,WoFTWkoOrciT0TEvRJcSTQ,1092


In [59]:
df_music = pd.DataFrame(df['MID'])

In [60]:
#map the ternary strings to list
def bitcountToRomBoolean(aid, ambfield):
    leftShift = aid//weights[ambfield]
    value__ = (leftShift)%3
    return value__
    
df_music['dj'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'dj'))
df_music['background_music'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'background_music'))
df_music['no_music'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'no_music'))

df_music['jukebox'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'jukebox'))
df_music['live'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'live'))
df_music['video'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'video'))

df_music['karaoke'] = df_music['MID'].apply(lambda x : bitcountToRomBoolean(x, 'karaoke'))

In [61]:
#remove duplicates
df_music.drop_duplicates(subset ="MID", 
                     keep = 'first', inplace = True) 
df_music

,MID,dj,background_music,no_music,jukebox,live,video,karaoke
18,1092,1,1,1,1,1,1,0
93,1119,1,1,1,2,1,1,0
139,1821,2,1,1,1,1,1,0
163,1257,1,2,0,1,1,2,0
206,1254,1,2,0,1,1,1,0
...,...,...,...,...,...,...,...,...
149989,1824,2,1,1,1,1,2,0
158094,1371,1,2,1,2,2,1,0
162480,1040,1,1,0,2,1,1,2
171005,1742,2,1,0,1,1,1,2
